Сначала нам надо скачать дату -- соберите как минимум 60 (30 положительных и 30 отрицательных) отзывов на похожие продукты (не надо мешать отзывы на отели с отзывами на ноутбуки) для составления " тонального словаря" (чем больше отзывов, тем лучше) и 10 отзывов для проверки качества. 2 балла в случае сбора путём парсинга, 1 - если найдете уже готовые данные или просто закопипастите без парсинга

In [165]:
import requests
import re
from pymystem3 import Mystem
m = Mystem()
from bs4 import BeautifulSoup
from sklearn.metrics import accuracy_score
from fake_useragent import UserAgent
ua = UserAgent(verify_ssl=False)

In [18]:
session = requests.session()

In [53]:
urls = ['https://petobzor.com/korm-dlya-koshek-royal-canin-otzyvy-i-razbor-sostava/',
       'https://petobzor.com/korm-dlya-koshek-brit-premium-otzyvy-i-razbor-sostava/',
       'https://petobzor.com/korm-dlya-koshek-carnilove-otzyvy-razbor-sostava/',
        'https://petobzor.com/korm-dlya-koshek-purina-one-otzyvy-i-razbor-sostava/',
       'https://petobzor.com/korm-dlya-koshek-friskies-otzyvy-i-razbor-sostava/']

In [61]:
comments = []
for url in urls:
    req = session.get(url, headers={'User-Agent': ua.random})
    page = req.text
    soup = BeautifulSoup(page, 'html.parser')
    for com in soup.find_all('li', {'class': 'review'}):
        comments.append(com)

In [118]:
good_comments = []
bad_comments = []


for com in comments:
    rating = re.search(r'\d,0 rating', str(com))
    text = re.findall(r'<p>(.+)</p>', str(com))
    if rating is not None:
        if rating[0][0] == '5' or rating[0][0] == '4':
            good_comments.append(text[0])
        else:
            bad_comments.append(text[0])

In [119]:
print(len(good_comments))
print(len(bad_comments))

37
43


In [125]:
print('Хороший:', good_comments[0])
print('\nПлохой:', bad_comments[0])

Хороший: Моя кошка два года питается кормом Royal Canin Sterilised 37, выглядит прекрасно, жалоб на здоровье нет. Поэтому я доверяю информации производителя о хорошем качестве. Цену корма дорогой не считаю, если брать 2 кг упаковку выходит дешевле чем 5 упаковок по 400 г. </p> <p>У данного производителя самая большая линейка что я видела, даже специальных кормов для стерилизованных кошек есть несколько (для животных разного возраста). </p> <p>Ранее пробовала перейти на обычный корм Royal Canin, кошка набрала лишний вес, поэтому вернулась назад. Но теперь знаю, что этот корм действительно помогает контролировать вес кошки.

Плохой: Купили кусочки в желе Royal Canin Kitten Instinctive. Корм премиум класса, это настраивало на позитивный результат. Реклама описывает его как разработанный с учетом всех потребностей животного, а состав разработан ветеринарными врачами. </p> <p>На практике все оказалось не так безоблачно. Котенок ел с удовольствием, я бы сказал с огромным удовольствием, но по

Токенизируйте слова, приведите их к нижнему регистру и к начальной форме (1 балл за токенизацию, 1 - за начальную форму)

In [145]:
good_tokens = []
bad_tokens = []
stop = [' ', '.', '!', ',', '(', ')', '\n', 'p', '</', '>', '><', '=', ' — ', '. ', ', ', ') ', '> <', '/> <', '/><']

for com in good_comments:
    tokens = m.lemmatize(com.lower())
    for token in tokens:
        if token not in stop:
            good_tokens.append(token)
            
for com in bad_comments:
    tokens = m.lemmatize(com.lower())
    for token in tokens:
        if token not in stop:
            bad_tokens.append(token)

In [144]:
print(len(good_tokens), len(bad_tokens), good_tokens[10:20], bad_tokens[1110:1120])

4586 3697 [', ', 'выглядеть', 'прекрасно', ', ', 'жалоба', 'на', 'здоровье', 'нет', '. ', 'поэтому'] ['маркетинг', ', ', 'но', 'не', 'на', 'корм', '> <', 'hr', '/> <', 'time']


Составьте 2 множества - в одном будут слова, которые встречаются только в положительных отзывах, а в другом - встречающиеся только в отрицательных. Попробуйте поиграть с частотностями и исключить шум (к примеру, выбросить слова, встречающиеся 1-2 раза) (3 балла) (если у вас получились пустые множества, уберите фильтр по частотности или увеличьте выборку)

In [147]:
raw_good_set = set(good_tokens)
raw_bad_set = set(bad_tokens)

In [151]:
good_set = raw_good_set.difference(raw_bad_set)
bad_set = raw_bad_set.difference(raw_good_set)

In [159]:
#убираем элементы, не являющиеся словами
remove_list = []
for elem in good_set:
    if not re.match(r'[a-zа-я]+', elem):
        remove_list.append(elem)
        
for elem in remove_list:
    good_set.remove(elem)

In [160]:
remove_list = []

for elem in bad_set:
    if not re.match(r'[a-zа-я]+', elem):
        remove_list.append(elem)
        
for elem in remove_list:
    bad_set.remove(elem)

Создайте функцию, которая будет определять, положительный ли отзыв или отрицательный в зависимости от того, какие слова встретились в нём, и посчитайте качество при помощи accuracy (1 - за коректно работающую функцию, 1 - за подсчёт accuracy)

In [187]:
def good_or_bad(text, good_set, bad_set):
    bad_score = 0
    good_score = 0
    result = ''
    tokens = m.lemmatize(text.lower())
    for token in tokens:
        if token in good_set:
            good_score +=1
        elif token in bad_set:
            bad_score +=1
    if bad_score > good_score:
        result = 'bad'
        print('Это отрицательный отзыв')
    else:
        result = 'good'
        print('Это положительный отзыв')
    return result

In [169]:
text = 'Ужасный корм! Дала попробовать его мужу, а он отравился и умер!'
good_or_bad(text, good_set, bad_set)

Это отрицательный отзыв


'bad'

In [170]:
text = 'Отличный корм, всем театром Куклачёва едим только его и прекрасно себя чувствуем.'
good_or_bad(text, good_set, bad_set)

Это положительный отзыв


'good'

In [177]:
#Соберём новые отзывы для проверки accuracy

check_url = 'https://petobzor.com/korm-dlya-koshek-mealfeel-otzyvy-razbor-sostava-tsena/'

check_comments = []
req = session.get(check_url, headers={'User-Agent': ua.random})
page = req.text
soup = BeautifulSoup(page, 'html.parser')
for com in soup.find_all('li', {'class': 'review'}):
    check_comments.append(com)
    
check_good_comments = []
check_bad_comments = []


for com in check_comments:
    rating = re.search(r'\d,0 rating', str(com))
    text = re.findall(r'<p>(.+)</p>', str(com))
    if rating is not None:
        if rating[0][0] == '5' or rating[0][0] == '4':
            check_good_comments.append(text[0])
        else:
            check_bad_comments.append(text[0])

In [185]:
y_true = ['good'] * len(check_good_comments) + ['bad'] * len(check_bad_comments)

In [188]:
y_pred = []
for com in check_good_comments:
    y_pred.append(good_or_bad(com, good_set, bad_set))
for com in check_bad_comments:
    y_pred.append(good_or_bad(com, good_set, bad_set))

Это положительный отзыв
Это положительный отзыв
Это положительный отзыв
Это положительный отзыв
Это положительный отзыв
Это положительный отзыв
Это отрицательный отзыв
Это отрицательный отзыв
Это положительный отзыв
Это положительный отзыв
Это положительный отзыв
Это положительный отзыв
Это положительный отзыв


In [189]:
accuracy_score(y_true, y_pred)

0.6153846153846154

Предложите как минимум 2 способа улучшить эту программу с помощью добавления к ней любых мулек - просто словами, писать улучшающий код не надо (1 балл)

0. Обучить нейросеточку на материале положительных и отрицательных отзывов, чтобы она научилась их различать.
1. Сравнивать не просто слова из + и - отзывов, а словосочетания (2-граммы, 3-граммы...)
2. Векторизовать все + и - отзывы. Вычислить "средний" вектор по + и по - отзывам. Новые отзывы тоже превращать в вектор и смотреть, к какому среднему вектору он ближе.
3. Собрать в пару тысяч раз больше отзывов.